In [1]:
# [START gmail_quickstart]
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import base64
from bs4 import BeautifulSoup
import regex as re
import spacy
import numpy as np
import json
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [2]:
creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
        creds = flow.run_local_server()
        # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
results = service.users().labels().list(userId='me').execute()

In [26]:
user_id = 'me'
threads = service.users().threads().list(userId=user_id).execute().get('threads', [])
results_msg = service.users().messages().list(userId=user_id,labelIds = ['INBOX']).execute()
messages = results_msg.get('messages', [])
threads

[{'id': '16acc54611713413',
  'snippet': 'Yes, I&#39;ll send you a meeting invite with a dial in number. Paul Mendoza, CEO Mobile 760-917-3753 SigParser pmendoza@sigparser.com Listen to podcasts? I was recently on the FutureTech Podcast',
  'historyId': '12951028'},
 {'id': '16adaddf8605b0c7',
  'snippet': '\ufeff May 2019 Edition Home | New Car | Pre-Owned Cars | Car News | Car Loan Avail Car Laon New Launches Jeep Compass Sport Plus Launched At Rs 15.99 Lakh Jeep Compass Sport Plus has been launched in India',
  'historyId': '12950173'},
 {'id': '16adaa0fc55395db',
  'snippet': 'You know those blissful vacation feels? This is like that pepperfry GET OUR APP pepperfry pepperfry Furniture Decor Lamps &amp; Lighting Mattresses &amp; Bedding More Holiday feels at home! coupon',
  'historyId': '12949717'},
 {'id': '16ada7ecc2b541ce',
  'snippet': 'Learn about 2 of the most critical libraries for distributing data science applications - plotly &amp; shiny Interactive Visualization with Plo

In [43]:
msg = service.users().messages().get(userId='me', id='16aca9286a68acb6').execute()
msg

{'id': '16aca9286a68acb6',
 'threadId': '16aca9286a68acb6',
 'labelIds': ['IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'],
 'snippet': '---------- Forwarded message --------- From: Donnelley Paul (BT/SAO-SBV BT/SSY-SAO) &lt;Paul.Donnelley@au.bosch.com&gt; Date: Fri, May 10, 2019 at 5:09 PM Subject: RE: Dangerous conditions for cycling',
 'historyId': '12951554',
 'internalDate': '1558176751000',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'shoumikgoswami@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a92:aa58:0:0:0:0:0 with SMTP id j85csp1849978ili;        Sat, 18 May 2019 03:52:41 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:660c:844:: with SMTP id f4mr6174172itl.139.1558176761799;        Sat, 18 May 2019 03:52:41 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1558176761; cv=none;        d=google.com; s=arc-20160816;        b=zgnAqb5iny2byaiXg

In [49]:
#define the storage variables
contact_list = []
name_track = []
email_track = []
phone_track = []
twitter_track = []
linkedin_track = []
facebook_track = []
other_contacts = []
contact_name = []
contact_num = []
contact_email = []

In [50]:
#Function to extract names from the string using spacy
def extract_name(string):
    names = []
    nlp = spacy.load('en')
    #nlp = spacy.load('en_core_web_sm')
    doc = nlp(string)
    ent_list = ['PERSON']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            names.append(ent.text)
    return np.unique(names)

In [52]:
def extract_org(string):
    org = []
    nlp = spacy.load('en')
    doc = nlp(string)
    ent_list = ['GPE','NORP','ORG','LOC','PRODUCT']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            org.append(ent.text)
    return np.unique(org)

In [51]:
#Function to extract Phone Numbers from string using regular expressions
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = np.unique(r.findall(string))
    return [re.sub(r'\D', '', number) for number in phone_numbers]

In [53]:
#Function to extract Email address from a string using regular expressions
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return np.unique(r.findall(string))
    #return r.findall(string)

In [54]:
def extract_twitter(string):
    r = re.compile(r'twitter.com/([^\s:]+)')
    return r.findall(string)

In [55]:
def extract_linkedin(string):
    r = re.compile(r'linkedin.com/([^\s:]+)')
    return r.findall(string)

In [56]:
def extract_facebook(string):
    r = re.compile(r'facebook.com/([^\s:]+)')
    return r.findall(string)

In [57]:
df = pd.DataFrame(msg['payload']['headers'])

In [106]:
contact_email.append(str(extract_email_addresses(str(df['value'].loc[df['name'] == 'From']))))
contact_name.append(str(extract_name(str(df['value'].loc[df['name'] == 'From']))))
contact_email.append(str(extract_email_addresses(str(df['value'].loc[df['name'] == 'To']))))
contact_name.append(str(extract_name(str(df['value'].loc[df['name'] == 'To']))))
contact_email.append(str(extract_email_addresses(str(df['value'].loc[df['name'] == 'Cc']))))
contact_name.append(str(extract_name(str(df['value'].loc[df['name'] == 'Cc']))))

In [58]:
#get content of the email
#body = msg['payload']['parts'][0]['parts'][0]['body']['data']
body = msg['payload']['parts'][0]['body']['data']
html_data = base64.urlsafe_b64decode(body)
soup = BeautifulSoup(html_data, 'html.parser')

In [78]:
data = msg['payload']['parts'][0]
#data_html = filter(lambda data: data['mimeType'] == 'text/plain', data)
data[data['mimeType'] == 'text/plain']['body']['data']

KeyError: True

In [70]:
print(data_html)

In [45]:
content = soup.get_text()
string = content.replace(',',' ')
string = string.replace('\n',' ')
string = string.replace('\r',' ')
string = string.replace('\\r\\n',' ')
string1 = string.replace('*','')

In [48]:
html_data

b'---------- Forwarded message ---------\r\nFrom: Donnelley Paul (BT/SAO-SBV BT/SSY-SAO) <Paul.Donnelley@au.bosch.com>\r\nDate: Fri, May 10, 2019 at 5:09 PM\r\nSubject: RE: Dangerous conditions for cycling Akuna (anticlockwise) this\r\nSunday 12th May\r\nTo: george lau <georgeandkim@gmail.com>, Adrian See <adriansee@hotmail.com>,\r\nAdrian Wong <adwong_au@yahoo.com>, Alex Perdomo <alejoperdomo68@gmail.com>,\r\nAndrew Linfoot <andrew.linfoot@gmail.com>, Andrew McPherson <\r\nandrewmcp@optusnet.com.au>, A&S <andsal@optusnet.com.au>, Andy Kuo <\r\nandy.kuo1@bayer.com>, Chi Ooi <chigooi@gmail.com>, Chris and Wendy Annable <\r\ncwannable@hotmail.com>, Courtenay Farquharson <\r\ncourtenay.farquharson@gmail.com>, Daniel Lu <mail2danlu@gmail.com>, David\r\nChen <dst.chen@gmail.com>, Dave Jackson <DAVID.A.JACKSON.1138@googlemail.com>,\r\nDavid Stanfield <david.stanfield@hotmail.com>, denis gradinscak <\r\ndgrad76@gmail.com>, Gavin Gillett <gavin@gillettproperties.com.au>, Ian\r\nPartis <ian_par

In [109]:
names= extract_name(string1)
org = extract_org(string1).tolist()
emails = extract_email_addresses(string1).tolist()
phone_num = extract_phone_numbers(string1)
twitter = extract_twitter(string1)
linkedin = extract_linkedin(string1)
facebook = extract_facebook(string1)

In [110]:
for i in range(len(emails)):
    name = []
    start_pos = []
    for match in re.finditer(emails[i], string1):
        start_pos.append(match.start())
    for j in range(len(start_pos)):
        end_pos = start_pos[j] + len(emails[i])
        name = extract_name(string1[start_pos[j]-50:start_pos[j]])
        if len(name) > 0:
            #contact = {'name':str(name),'email':emails[i]}
            #contact_test.append(contact)
            if (len(extract_phone_numbers(string1[start_pos[j]-100:start_pos[j]]))>0):
                contact_name.append(str(name))
                contact_email.append(emails[i])
                contact_num.append(extract_phone_numbers(string1[start_pos[j]-100:start_pos[j]]))
            elif (len(extract_phone_numbers(string1[end_pos:end_pos + 150]))>0):
                contact_name.append(str(name))
                contact_email.append(emails[i])
                contact_num.append(extract_phone_numbers(string1[end_pos:end_pos + 150]))
            else:
                contact_name.append(str(name))
                contact_email.append(emails[i])
                contact_num.append('null')

In [111]:
contact_list_new = pd.DataFrame()
contact_list_new['contact_name'] = contact_name
contact_list_new['contact_email'] = contact_email
#contact_list_new['contact_num'] = contact_num

In [112]:
contact_list_new.drop_duplicates(subset='contact_email')

,contact_name,contact_email
0,['Paul Mendoza'],['pmendoza@sigparser.com']
1,[],['shoumikgoswami@gmail.com']
2,[],[]


In [113]:
contact_list = contact_list_new.drop_duplicates(subset='contact_email').reset_index()

In [ ]:
#name_track = contact_list['contact_name']
#email_track = contact_list['contact_email']
#for i in names:
#    if i not in name_track:
#        contact = {'name':i,'email':''}
#        other_contacts.append(contact)
#for i in emails:
#    if i not in email_track:
#        contact = {'name':'','email':i}
#        other_contacts.append(contact)

In [114]:
contact_list = contact_list_new.drop_duplicates(subset='contact_email').reset_index().to_dict('index')

In [115]:
response = {'contact': contact_list,
           #'other_contact': other_contacts,
           #'organisation': org,
           'phone number': phone_num,
           'twitter handle': twitter,
           'linkedin handle': linkedin,
           'facebook handle': facebook}

In [116]:
response

{'contact': {0: {'index': 0,
   'contact_name': "['Paul Mendoza']",
   'contact_email': "['pmendoza@sigparser.com']"},
  1: {'index': 1,
   'contact_name': '[]',
   'contact_email': "['shoumikgoswami@gmail.com']"},
  2: {'index': 2, 'contact_name': '[]', 'contact_email': '[]'}},
 'phone number': ['7609173753'],
 'twitter handle': [],
 'linkedin handle': [],
 'facebook handle': []}

In [ ]:
json_resp = json.dumps(response)

In [ ]:
import requests
import json

url = "https://api.sigparser.com/api/Email"

payload = json.dumps( {"from_address": "jsmith@example.com","from_name": "John Smith", "plainbody": content, "htmlbody": soup } )
headers = {
    'content-type': "application/json",
    'x-api-key': "Jcrw4cwnJJ4zhLzgQMWrt5cpgbK8Andu7NOVPxVB",
    'cache-control': "no-cache"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)\


In [ ]:
#spam classifier
from keras.models import load_model
import pandas as pd
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder

In [ ]:
model = load_model('spam_detector_enron_model.h5')

In [ ]:
text = str(content)

In [ ]:
num_max = 4000

def tokenize(row):
    "tokenize the text using default space tokenizer"
    text=row
    lines=(line for line in text.split('\n') )
    tokenized=""
    for sentence in lines:
        tokenized+= " ".join(tok for tok in sentence.split())
    return tokenized

def train_tf_idf_model(texts):
    "train tf idf model "
    tok = Tokenizer(num_words=num_max)
    tok.fit_on_texts(texts)
    return tok

def prepare_model_input(tfidf_model,dataframe,mode='tfidf'):
    
    le = LabelEncoder()
    sample_texts = list(dataframe)
    sample_texts = [' '.join(x.split()) for x in sample_texts]
    if mode=='tfidf':
        sample_texts=tfidf_model.texts_to_matrix(sample_texts,mode='tfidf')
    else:
        sample_texts=tfidf_model.texts_to_matrix(sample_texts)
    return sample_texts

def predict(data):
    result=model.predict(data)
    prediction = [round(x[0]) for x in result]
    return prediction

In [ ]:
tokenized_text = tokenize(text)
texts=list(tokenized_text)
tfidf_model=train_tf_idf_model(texts)
mat_texts =prepare_model_input(tfidf_model,tokenized_text, mode='')
prediction = predict(mat_texts)
mean_pred = sum(prediction)/len(prediction)
print(mean_pred)

In [3]:
#define the storage variables
contact_list = []
name_track = []
email_track = []
phone_track = []
twitter_track = []
linkedin_track = []
facebook_track = []
other_contacts = []
contact_name = []
contact_num = []
contact_email = []

In [6]:
#Function to extract names from the string using spacy
def extract_name(string):
    names = []
    nlp = spacy.load('en')
    #nlp = spacy.load('en_core_web_sm')
    doc = nlp(string)
    ent_list = ['PERSON']
    for ent in doc.ents:
        if(ent.label_ in ent_list):
            #print(ent.text, ent.label_)
            names.append(ent.text)
    return np.unique(names)

#Function to extract Phone Numbers from string using regular expressions
def extract_phone_numbers(string):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = np.unique(r.findall(string))
    return [re.sub(r'\D', '', number) for number in phone_numbers]

#Function to extract Email address from a string using regular expressions
def extract_email_addresses(string):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return np.unique(r.findall(string))
    #return r.findall(string)
    
def extract_twitter(string):
    r = re.compile(r'twitter.com/([^\s:]+)')
    return r.findall(string)

def extract_linkedin(string):
    r = re.compile(r'linkedin.com/([^\s:]+)')
    return r.findall(string)

def extract_facebook(string):
    r = re.compile(r'facebook.com/([^\s:]+)')
    return r.findall(string)

def find_nearby(email, email_body):
    start_pos = []
    number = []
    twitter = []
    facebook = []
    linkedin = []
    for match in re.finditer(email, email_body):
        start_pos.append(match.start())
    for j in range(len(start_pos)):
        end_pos = start_pos[j] + len(email)
        number.append(extract_phone_numbers(email_body[start_pos[j]-100:start_pos[j]]))
        number.append(extract_phone_numbers(email_body[end_pos:end_pos + 100]))
        twitter.append(extract_twitter(email_body[start_pos[j]-50:start_pos[j]]))
        twitter.append(extract_twitter(email_body[end_pos:end_pos + 100]))
        facebook.append(extract_facebook(email_body[start_pos[j]-50:start_pos[j]]))
        facebook.append(extract_facebook(email_body[end_pos:end_pos + 100]))
        linkedin.append(extract_linkedin(email_body[start_pos[j]-50:start_pos[j]]))
        linkedin.append(extract_linkedin(email_body[end_pos:end_pos + 100]))
    return number,twitter,linkedin

def text_cleaning(text):
#     text = [word for word in text.split(' ') if word.lower() not in stopwords.words('english')]
#     text =''.join(nopunc)
    
    # removing non unicode
    text = unicodedata.normalize("NFKD", text)
    #Removing commas and spaces in the resume for an effecient check
#     text = text.replace(',',' ')
#     text = text.replace('\\n',' ')
#     text = text.replace('\\r',' ')
#     text = text.replace('\\r\\n',' ')
#     text = text.replace('*','')
    
    bad_characters = ['\r', '\n', '\\r\\n', ',', '*', ';']
    nopunc = [" " if char in bad_characters else char for char in text]
    text =''.join(nopunc)
    text = re.sub(' +', ' ', text)
    bad_words= ['&gt;', '&gt', '&lt']
    text = text.split(' ')
    text = ['' if word in bad_words else word for word in text]
    text =' '.join(text)
#     # removing punctuation
#     nopunc = [char for char in text if char not in string.punctuation]
#     text =''.join(nopunc)
    # removing stop words
    
    return text

def find_correct_name(names, email_addr):
    name_counter={name.title():0 for name in names}
    for name in name_counter:
        for item in name.split(' '):
            name_counter[name] += len(re.findall(item.lower(), email_addr))
    print(name_counter)
    name_counter = sorted(name_counter.items(), key=operator.itemgetter(1), reverse= True)
    if name_counter[0][1]>0:
        return name_counter[0][0]
#     return max(name_counter.items(), key=operator.itemgetter(1))[0]

In [2]:
def response(sender_email,sender_name,email_body):
    
    contact_name.append(sender_name)
    contact_email.append(sender_email)
    
    html_data = email_body
    soup = BeautifulSoup(html_data, 'html.parser')
    content = soup.get_text()
    content = text_cleaning(content)
    
    for line in content.split('>'):
        email_addrs = extract_email_addresses(line)
        name=None
        if len(email_addrs) == 1:
            names = extract_name(line)
            if len(names)>0:
                name= find_correct_name(names, email_addrs[0])
    #         elif len(names) == 1:
    #             name = names[0]
            else:
                pass
        else:
            pass
        if name is not None:
            contact_name.append(name)
            contact_email.append(email_addrs[0])
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-909a1e1a277f>, line 2)